In [1]:
from qiskit import *
from qiskit.providers.fake_provider import FakeManila, FakeKolkata, FakeSherbrooke
from qiskit_ibm_provider import IBMProvider

import mthree
from mthree.utils import *
from mthree.generators.random import RandomGenerator, RandomComplimentGenerator
from mthree.generators import HadamardGenerator
from mthree._helpers import system_info

from qiskit.primitives import BackendEstimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session, Options

from mthree.twirling.tw_calibrations import Tw_Calibration
from mthree.twirling.tw_circuits import Tw_Circuit
from mthree.twirling.tw_utils import vals_from_dict
from mthree.twirling.tw_utils import convert_to_probabilities, total_variation_distance, marginalize_calibration_counts

In [2]:
provider = IBMProvider()
backend = FakeKolkata()

In [3]:
def donothing(N):
    qc = QuantumCircuit(N, N)
    for i in range(N):
        qc.measure(i,i)
    trans_qc = transpile(qc, backend, optimization_level=3, seed_transpiler=12345)
    return trans_qc

def GHZ(N):
    qc = QuantumCircuit(N, N)
    qc.h(0)
    for i in range(1,N):
        qc.cx(0,i)
    for i in range(N):
        qc.measure(i,i)
    trans_qc = transpile(qc, backend, optimization_level=3, seed_transpiler=12345)
    return trans_qc

def m3circuit(N):
    qc = QuantumCircuit(N, N)
    qc.x(range(N))
    qc.h(range(N))

    for kk in range(N // 2, 0, -1):
        qc.ch(kk, kk - 1)
    for kk in range(N // 2, N - 1):
        qc.ch(kk, kk + 1)
    for i in range(N):
        qc.measure(i,i)
    trans_qc = transpile(qc, backend, optimization_level=3, seed_transpiler=12345)
    return trans_qc

def untranspiled_donothing(N):
    qc = QuantumCircuit(N, N)
    for i in range(N):
        qc.measure(i,i)
    return qc

def untranspiled_GHZ(N):
    qc = QuantumCircuit(N, N)
    qc.h(0)
    for i in range(1,N):
        qc.cx(0,i)
    for i in range(N):
        qc.measure(i,i)
    return qc

def untranspiled_m3circuit(N):
    qc = QuantumCircuit(N, N)
    qc.x(range(N))
    qc.h(range(N))

    for kk in range(N // 2, 0, -1):
        qc.ch(kk, kk - 1)
    for kk in range(N // 2, N - 1):
        qc.ch(kk, kk + 1)
    for i in range(N):
        qc.measure(i,i)
    return qc

def ManyParallel_uint2bits(in_intAr,Nbits):
    ''' convert (numpyarray of uint => array of Nbits bits) for many bits in parallel'''
    inSize_T= in_intAr.shape
    in_intAr_flat=in_intAr.flatten()
    out_NbitAr= np.zeros((len(in_intAr_flat),Nbits))
    for iBits in range(Nbits):
        out_NbitAr[:,iBits]= (in_intAr_flat>>iBits)&1
    out_NbitAr= out_NbitAr.reshape(inSize_T+(Nbits,))
    out_NbitAr = np.flip(out_NbitAr)
    return out_NbitAr.astype('uint8')

class CompleteGenerator:
    """Complete bit-array generator with every bit array"""

    def __init__(self, num_qubits, seed=None):
        """Generator of random arrays corresponding to random x-gates on
        qubits for TexMex mitigation

        Parameters:
            num_qubits (int): Number of qubits
            seed (int): seed for RNG, default=None

        Attributes:
            num_qubits (int): Number of qubits / length of arrays
            length (int): Total number of generated arrays, default=16
            seed (int): Seed used for RNG
        """
        self.name = "random"
        self.seed = seed
        if self.seed is None:
            self.seed = np.random.randint(0, np.iinfo(np.int32).max)
        self._RNG = np.random.default_rng(seed=self.seed)
        self.num_qubits = num_qubits
        self.length = 2**num_qubits
        self._iter_index = 0

    def __iter__(self):
        self._RNG = np.random.default_rng(seed=self.seed)
        self._iter_index = 0
        return self

    def __next__(self):
        if self._iter_index < self.length:
            self._iter_index += 1
            return ManyParallel_uint2bits(np.asarray(self._iter_index),self.num_qubits)
        else:
            raise StopIteration
        
def all_z_op(qc, num_backend_qubits):
    operator_string = list(num_backend_qubits*'I')
    for key,value in final_measurement_mapping(qc).items():
        little_endian_index = num_backend_qubits-1-value
        operator_string[little_endian_index] = 'Z'
    operator_string = "".join(operator_string)
    op = SparsePauliOp.from_list([(operator_string, 1)])
    return op

In [4]:
N=8
qc = GHZ(N)
#qc = m3circuit(N)
measurement_map = final_measurement_mapping(qc)
qubits_to_calibrate = vals_from_dict(measurement_map)
print(qubits_to_calibrate)

[24, 16, 20, 22, 19, 25, 26, 23]


/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [5]:
tw_circuit = Tw_Circuit(backend, circuit=qc, generator=HadamardGenerator)
tw_circuit.tw_data_from_backend(shots=2**13)
tw_counts = tw_circuit.to_untwirled_data()

tw_cal = Tw_Calibration(backend, qubits=qubits_to_calibrate, generator=HadamardGenerator)
tw_cal.tw_data_from_backend(shots=2**13)
tw_calib_counts = tw_cal.to_untwirled_data()

tw_results = tw_cal.tw_expval_and_stddev(counts=tw_circuit.to_untwirled_data(), operator=N*'Z', mapping=measurement_map)  
print(tw_results)

(0.8958569807037458, 0.07903952593635666)


In [6]:
complete_tw_circuit = Tw_Circuit(backend, circuit=qc, generator=CompleteGenerator)
complete_tw_circuit.tw_data_from_backend(shots=2**16)
complete_tw_counts = complete_tw_circuit.to_untwirled_data()
ground_truth = convert_to_probabilities(complete_tw_counts)

mit = mthree.M3Mitigation(backend)
mit.cals_from_system(measurement_map, shots=2**16)

In [7]:
untw_counts = execute(qc, backend, shots=8192).result().get_counts()

tw_circuit = Tw_Circuit(backend, circuit=qc, generator=HadamardGenerator)
tw_circuit.tw_data_from_backend(shots=8192)
tw_counts = tw_circuit.to_untwirled_data()

mit_untw_counts = mit.apply_correction(untw_counts, qubits=qubits_to_calibrate, return_mitigation_overhead=True)
mit_tw_counts = mit.apply_correction(tw_counts, qubits=qubits_to_calibrate, return_mitigation_overhead=True)

mit_untw_counts.expval_and_stddev(N*'Z'), mit_tw_counts.expval_and_stddev(N*'Z')
#total_variation_distance(mit_untw_counts, ground_truth), total_variation_distance(mit_tw_counts, ground_truth)

((0.9164008177503066, 0.013159833886287598),
 (0.9244374199472409, 0.013143628315831195))

In [8]:
trials = 100
untw_tvds = []
tw_tvds = []

for _ in range(trials):
    untw_counts = execute(qc, backend, shots=8192).result().get_counts()

    tw_circuit = Tw_Circuit(backend, circuit=qc, generator=HadamardGenerator)
    tw_circuit.tw_data_from_backend(shots=8192)
    tw_counts = tw_circuit.to_untwirled_data()

    mit_untw_counts = mit.apply_correction(untw_counts, qubits=qubits_to_calibrate, return_mitigation_overhead=True)
    mit_tw_counts = mit.apply_correction(tw_counts, qubits=qubits_to_calibrate, return_mitigation_overhead=True)
    
    untw_tvds.append(total_variation_distance(mit_untw_counts, ground_truth))
    tw_tvds.append(total_variation_distance(mit_tw_counts, ground_truth))

np.average(untw_tvds), np.average(tw_tvds)

(0.07126040869177229, 0.07022352614867415)

In [9]:
from qiskit import Aer
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
import qiskit_aer.noise as noise

# Build noise model from backend properties
backend = FakeKolkata()
noise_model = NoiseModel.from_backend(backend)
num_backend_qubits = system_info(backend)["num_qubits"]

# Get coupling map from backend
# coupling_map = backend.configuration().coupling_map

# Error probabilities
prob_1 = 0.001  # 1-qubit gate
prob_2 = 0.01   # 2-qubit gate
prob_3 = 0.05  # bit flip readout
readout_probabilities = [[1-prob_3, prob_3], [prob_3, 1-prob_3]]   # measurement error 

# Depolarizing and Readout quantum errors
error_1 = noise.depolarizing_error(prob_1, 1)
error_2 = noise.depolarizing_error(prob_2, 2)
error_3 = noise.ReadoutError(readout_probabilities)

# Add errors to noise model
noise_model = noise.NoiseModel()
#noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3'])
#noise_model.add_all_qubit_quantum_error(error_2, ['cx'])
noise_model.add_all_qubit_readout_error(error_3)

# Get basis gates from noise model
basis_gates = noise_model.basis_gates

# Set options for trex
options = Options()
options.simulator = {
    "noise_model": noise_model,
    "seed_simulator": 42
}
options.execution.shots = 2**13
options.optimization_level = 0 # no optimization
options.resilience_level = 1 # M3 for Sampler and T-REx for Estimator

In [19]:
N=20
#qc = untranspiled_GHZ(N)
qc = untranspiled_donothing(N)

# Perform a noise simulation
backend = AerSimulator(noise_model=noise_model, basis_gates=basis_gates)
transpiled_circuit = transpile(qc, backend)
result = backend.run(transpiled_circuit, shots=2**16).result()
counts = result.get_counts()
print(expval(items=counts, exp_ops=N*'Z'))
#plot_histogram(counts)
print(np.sqrt(2*np.log(4/0.05)/2**13))

0.129241943359375
0.03270826686567064


In [11]:
measurement_map = final_measurement_mapping(transpiled_circuit)
qubits_to_calibrate = vals_from_dict(measurement_map)

mit = mthree.M3Mitigation(backend)
mit.cals_from_system(measurement_map, shots=2**16)

In [12]:
untw_counts = execute(transpiled_circuit, backend, shots=2**13).result().get_counts()

tw_circuit = Tw_Circuit(backend, circuit=transpiled_circuit, generator=HadamardGenerator)
tw_circuit.tw_data_from_backend(shots=2**13)
tw_counts = tw_circuit.to_untwirled_data()

m3_untw_counts = mit.apply_correction(untw_counts, qubits=qubits_to_calibrate, return_mitigation_overhead=True)
m3_tw_counts = mit.apply_correction(tw_counts, qubits=qubits_to_calibrate, return_mitigation_overhead=True)

m3_untw_counts.expval_and_stddev(N*'Z'), m3_tw_counts.expval_and_stddev(N*'Z')

((0.9036838925083749, 0.06913109358580186),
 (0.9204946819457211, 0.06901661035317896))

In [13]:
tw_cal = Tw_Calibration(backend, qubits=qubits_to_calibrate, generator=HadamardGenerator)
tw_cal.tw_data_from_backend(shots=2**16)
tw_calib_counts = tw_cal.to_untwirled_data()
calib_map = tw_cal.physical_to_bit_mapping

In [14]:
tw_circuit = Tw_Circuit(backend, circuit=transpiled_circuit, generator=HadamardGenerator)
tw_circuit.tw_data_from_backend(shots=2**13)
tw_counts = tw_circuit.to_untwirled_data()

tw_results = tw_cal.tw_expval_and_stddev(counts=tw_circuit.to_untwirled_data(), operator=N*'Z', mapping=measurement_map)  
print(tw_results)

(1.0447011551983927, 0.40263407388431305)


In [15]:
trials = 100
tw_vals = []

for _ in range(trials):
    tw_circuit = Tw_Circuit(backend, circuit=transpiled_circuit, generator=HadamardGenerator)
    tw_circuit.tw_data_from_backend(shots=8192)
    tw_counts = tw_circuit.to_untwirled_data()

    tw_vals.append(tw_cal.tw_expval(counts=tw_counts, operator=N*'Z', mapping=measurement_map))

print(np.average(tw_vals), np.std(tw_vals))

1.0029331993972879 0.0820083885543406


In [16]:
trials = 100
m3_untw_vals = []
m3_tw_vals = []

for _ in range(trials):
    untw_counts = execute(transpiled_circuit, backend, shots=8192).result().get_counts()

    tw_circuit = Tw_Circuit(backend, circuit=transpiled_circuit, generator=HadamardGenerator)
    tw_circuit.tw_data_from_backend(shots=8192)
    tw_counts = tw_circuit.to_untwirled_data()

    mit_untw_counts = mit.apply_correction(untw_counts, qubits=qubits_to_calibrate, return_mitigation_overhead=True)
    mit_tw_counts = mit.apply_correction(tw_counts, qubits=qubits_to_calibrate, return_mitigation_overhead=True)
    
    m3_untw_vals.append(mit_untw_counts.expval(N*'Z'))
    m3_tw_vals.append(mit_tw_counts.expval(N*'Z'))

print(np.average(m3_untw_vals), np.std(m3_untw_vals))
print(np.average(m3_tw_vals), np.std(m3_tw_vals))

0.8969296495702881 0.044260277833133
0.8974904606068741 0.05011601166611282


In [17]:
trials=100

with Session(backend="ibmq_qasm_simulator"):
    # include the noise model with T-REx
    estimator = Estimator(options=options)
    job = estimator.run(circuits=trials*[transpiled_circuit], observables=trials*[N*'Z'])
    result = job.result()
    metadata = result.metadata
    estimated_variance = metadata[0]["variance"] / metadata[0]["shots"]
    print(estimated_variance)
    trex_expval_and_stddev = np.average(result.values), np.std(result.values)
    print(trex_expval_and_stddev)

0.009121739191942055
(1.0598936170212767, 0.08848167232658621)


In [18]:
def D(p,q):
    return p*np.log(p/q)+(1-p)*np.log((1-p)/(1-q))

alpha=0.0327
p=0.5
d_target = np.log(4/0.05)/2**13
alpha_hoeff = np.sqrt(2*d_target)
print(alpha_hoeff)

D(p+alpha/2,p)/d_target

0.03270826686567064


0.9996727741335084